### > 모듈

In [1]:
### 해시태그 전처리 모듈

def PreprocessingHashtags(path):
    
    import pandas as pd
    import re
    
    # 데이터 불러오기
    data = pd.read_csv(path).iloc[:, 1:]
    data = data[data['hashtags'].notnull()] # 해시태그 nan 제거
    data = data.drop_duplicates('image_url') # 중복 행 제거
    
    # 한글, 영어 외 제거 후 우물 정(#) 기준으로 분리
    p = re.compile(r'[가-힣a-zA-Z#]+')
    data['hashtags_splitted'] = data['hashtags'].apply(lambda x: ''.join(p.findall(str(x))).split('#'))
    
    # 빈 해시태그 제거
    data['hashtags_completed'] = ''
    for i in range(len(data)):
        ls = [word for word in data.iloc[i]['hashtags_splitted'] if word!='']
        data['hashtags_completed'].iloc[i] = ls
        
    # 컬럼 삭제
    data.drop(['hashtags', 'hashtags_splitted'], axis=1, inplace=True)
        
    # 컬럼명 변경
    data.rename({'hashtags_completed':'hashtags'}, axis=1, inplace=True) 
    
    # 인덱스 reset
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)

    return data

In [2]:
### 출현빈도가 1회뿐인 해시태그 제거 모듈
### 시간이 오래걸려서 전처리 모듈과 따로 분리했습니다.

def PreprocessingHashtags_deletefreq(data):
    
    import pandas as pd
    
    # 데이터 가져오기
    data = data
    
    # 출현빈도가 1회뿐인 해시태그 제거
    hashtags_list = []
    for x in range(len(data.hashtags)):
        for y in range(len(data.hashtags[x])):
            hashtags_list.append(data.hashtags[x][y])
            
    hashtags_set = set(hashtags_list)
    hashtags_count = [hashtags_list.count(i) for i in hashtags_set]
    hashtags_dict = dict(zip(hashtags_set, hashtags_count))

    hashtags_df = pd.DataFrame()
    hashtags_df['name'] = hashtags_dict.keys()
    hashtags_df['count'] = hashtags_dict.values()
    
    hashtags_df = hashtags_df[hashtags_df['count'] >= 2]
    hashtags_df = hashtags_df.reset_index().drop('index', axis=1)
    
    # 컬럼 변경 완료
    new_hashtags = []
    for x in range(len(data.hashtags)):
        temp = []
    
        for y in range(len(data.hashtags[x])):    
            if data.hashtags[x][y] in hashtags_list:
                temp.append(data.hashtags[x][y])
        new_hashtags.append(temp)
        
    data['hashtags'] = new_hashtags
    
    return data

In [3]:
### 토큰화 모듈

def Tokenizer(data):
    
    from konlpy.tag import Okt
    
    # 데이터 가져오기
    data = data
    new_hashtags = data.hashtags.copy()
    
    # 토큰화
    for i in range(len(new_hashtags)):
        new_hashtags[i] = ' '.join(new_hashtags[i])
    
    okt = Okt()
    tokenized = []

    for sentence in new_hashtags:
        tokens = okt.morphs(sentence)
        tokenized.append(tokens)
    
    # 중복된 토큰 제거
    new_tokenized = []
    
    for x in range(len(tokenized)):
        temp = []
    
        for y in range(len(tokenized[x])):
            if tokenized[x][y] not in temp:
                temp.append(tokenized[x][y])
    
        new_tokenized.append(temp)
    
    return new_tokenized

In [4]:
### Word2Vec 학습 모듈

def Word2Vec(tokenized, min_count=1, workers=8, size=70, window=40, sg=1, iter=10):
    
    from gensim.models.word2vec import Word2Vec
    
    # Word2Vec 학습

    """
    sentences: 학습할 문장
    min_count : 임베딩할 단어의 최소 빈도수
    workers: 병렬 처리 스레드 수
    size: word vector의 차원(임베딩 사이즈)
    window: 윈도우 크기
    sg: skip-gram 사용여부(1: 사용, other: CBOW)
    iter: 학습횟수
    """
    
    model = Word2Vec(sentences = tokenized,
                 min_count = min_count,
                 workers = workers,
                 size = size,
                 window = window,
                 sg = sg,
                 iter=iter)
    
    # 학습이 완료 되면 필요없는 메모리를 unload
    model.init_sims(replace=True)
    
    return model

### > 사용예시

In [5]:
data = PreprocessingHashtags('data.csv')
data = PreprocessingHashtags_deletefreq(data)
tokenized = Tokenizer(data)
model = Word2Vec(tokenized, iter=1)

In [6]:
t = '여행'
model.wv.most_similar(t,topn=5)

[('감성', 0.9832465648651123),
 ('미치다', 0.9708598256111145),
 ('힐링', 0.9660876989364624),
 ('에', 0.9643505811691284),
 ('food', 0.9623520970344543)]

In [7]:
t = '일상'
model.wv.most_similar(t,topn=5)

[('소통', 0.9836547374725342),
 ('인스타', 0.9826146364212036),
 ('그램', 0.9819351434707642),
 ('스타', 0.9777668714523315),
 ('모델', 0.9748936891555786)]

In [8]:
# model_name 지정 후 모델 저장 
model.save('model/model_name')

In [9]:
# 저장된 모델 사용하기
from gensim.models.word2vec import Word2Vec
model_1 = Word2Vec.load('model/model_1')
model_full = Word2Vec.load('model/model_full') # 형태소 단위가 아니라 해시태그 하나를 토큰으로 두고 학습시킨 모델

In [11]:
t = '여행'
print('model_1: \n {}'.format(model_1.wv.most_similar(t,topn=10)), '\n')
print('model_full: \n {}'.format(model_full.wv.most_similar(t,topn=10)))

model_1: 
 [('바다', 0.8299013376235962), ('힐링', 0.7942224740982056), ('미치다', 0.7778294086456299), ('제주도', 0.7431781888008118), ('커피', 0.742461085319519), ('야경', 0.7047562599182129), ('제주', 0.7015856504440308), ('사진', 0.6972116231918335), ('드라이브', 0.6897653937339783), ('데이트', 0.6890974044799805)] 

model_full: 
 [('여행스타그램', 0.8545773029327393), ('바다', 0.8091214299201965), ('travel', 0.7684853076934814), ('드라이브', 0.7474426031112671), ('제주도', 0.7408050298690796), ('떠나요', 0.7337548732757568), ('휴가', 0.7317897081375122), ('국내여행', 0.723529577255249), ('포항여행', 0.7225464582443237), ('세부', 0.7187522053718567)]
